# Mejora de imagen en baja iluminación (Gamma + CLAHE)

Este notebook te permite **subir una imagen**, **cargarla** y **aplicar** un flujo de mejora:
1. **Corrección gamma** (para iluminar sombras).
2. **CLAHE** en el canal **L** (LAB) para aumentar contraste local.
3. (Opcional) ajuste fino de contraste/brillo.

> Ejecuta las celdas en orden. Si trabajas en **Google Colab**, no necesitas instalar nada más que lo que se indica en la primera celda.


In [ ]:
# Colab/entorno: instalar dependencias si hiciera falta
!pip -q install opencv-python scikit-image matplotlib ipywidgets


In [ ]:
from google.colab import files
print('Selecciona tu fotografía (jpg/png)...')
uploaded = files.upload()  # sube archivo desde tu equipo

# Tomar el primer archivo subido
if len(uploaded) == 0:
    raise RuntimeError('No se subió ninguna imagen.')
IMG_PATH = list(uploaded.keys())[0]
print(f'Usando: {IMG_PATH}')


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def show_bgr(img_bgr, title='', size=(6,6)):
    plt.figure(figsize=size)
    plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    plt.title(title); plt.axis('off')
    plt.show()

def adjust_gamma(image_bgr, gamma=0.6):
    inv_gamma = 1.0 / gamma
    table = np.array([((i/255.0)**inv_gamma) * 255 for i in np.arange(256)]).astype('uint8')
    return cv2.LUT(image_bgr, table)

def enhance_low_light(image_bgr, gamma=0.6, clip_limit=2.0, tile_grid=(8,8), alpha=1.2, beta=5):
    img_gamma = adjust_gamma(image_bgr, gamma=gamma)
    lab = cv2.cvtColor(img_gamma, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid)
    L_eq = clahe.apply(L)
    lab_eq = cv2.merge([L_eq, A, B])
    img_eq = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)
    img_final = cv2.convertScaleAbs(img_eq, alpha=alpha, beta=beta)
    return img_gamma, img_eq, img_final


In [ ]:
import cv2, matplotlib
img_bgr = cv2.imread(IMG_PATH)
if img_bgr is None:
    raise FileNotFoundError(f'No se pudo leer la imagen: {IMG_PATH}')

gamma = 0.6
clip_limit = 2.0
tile_grid = (8, 8)
alpha, beta = 1.2, 5

img_gamma, img_clahe, img_final = enhance_low_light(
    img_bgr, gamma=gamma, clip_limit=clip_limit, tile_grid=tile_grid, alpha=alpha, beta=beta
)

import matplotlib.pyplot as plt
plt.figure(figsize=(18,6))
plt.subplot(1,3,1); plt.imshow(cv2.cvtColor(img_gamma, cv2.COLOR_BGR2RGB)); plt.title(f'Gamma γ={gamma}'); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(cv2.cvtColor(img_clahe, cv2.COLOR_BGR2RGB)); plt.title(f'CLAHE (clip={clip_limit}, grid={tile_grid})'); plt.axis('off')
plt.subplot(1,3,3); plt.imshow(cv2.cvtColor(img_final, cv2.COLOR_BGR2RGB)); plt.title(f'Final (α={alpha}, β={beta})'); plt.axis('off')
plt.tight_layout(); plt.show()

cv2.imwrite('mejorada_gamma.jpg', img_gamma)
cv2.imwrite('mejorada_clahe.jpg', img_clahe)
cv2.imwrite('mejorada_final.jpg', img_final)
print('Guardado: mejorada_gamma.jpg, mejorada_clahe.jpg, mejorada_final.jpg')


In [ ]:
import ipywidgets as widgets
from IPython.display import display

gamma_slider = widgets.FloatSlider(value=0.6, min=0.3, max=1.5, step=0.05, description='γ')
clip_slider  = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='clip')
alpha_slider = widgets.FloatSlider(value=1.2, min=0.8, max=2.0, step=0.05, description='α')
beta_slider  = widgets.IntSlider(value=5, min=-20, max=40, step=1, description='β')
grid_slider  = widgets.IntSlider(value=8, min=4, max=16, step=1, description='grid')

def update(_):
    g = gamma_slider.value
    cl = clip_slider.value
    tg = (grid_slider.value, grid_slider.value)
    a = alpha_slider.value
    b = beta_slider.value
    _, _, out = enhance_low_light(img_bgr, gamma=g, clip_limit=cl, tile_grid=tg, alpha=a, beta=b)
    show_bgr(out, title=f'γ={g}, clip={cl}, grid={tg}, α={a}, β={b}', size=(8,6))

for w in [gamma_slider, clip_slider, grid_slider, alpha_slider, beta_slider]:
    w.observe(update, names='value')

display(gamma_slider, clip_slider, grid_slider, alpha_slider, beta_slider)
update(None)
